# Resolving Strategies

* Example on how to use resolving strategies


In [1]:
import os
import json

from kgforge.core import KnowledgeGraphForge
from kgforge.specializations.resources import Dataset

In [2]:
import getpass
TOKEN = getpass.getpass()

In [3]:
BUCKET = "dke/kgforge"
forge = KnowledgeGraphForge("../use-cases/prod-forge-nexus.yml",
 bucket=BUCKET,
 token=TOKEN
 )

### List available resolvers

In [4]:
forge.resolvers(output='dict')

{'agent': {'agents': {'bucket': 'bbp/agents', 'filters': None}},
 'ontology': {'terms': {'bucket': 'neurosciencegraph/datamodels',
   'filters': None},
  'CellType': {'bucket': 'neurosciencegraph/datamodels',
   'filters': {'subClassOf*.id': 'BrainCellType'}},
  'BrainRegion': {'bucket': 'neurosciencegraph/datamodels',
   'filters': {'subClassOf*.id': 'BrainRegion'}},
  'Species': {'bucket': 'neurosciencegraph/datamodels',
   'filters': {'subClassOf*.id': 'Species'}}}}

In [5]:
case_insensitive = forge.resolve('Pv+', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH')

In [6]:
exact = forge.resolve('PV+', scope='ontology', strategy='EXACT_MATCH')

In [7]:
assert case_insensitive == exact

### Try with a similar string, but other characters

In [8]:
similar = forge.resolve('pv-', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH')

In [9]:
similar1 = forge.resolve('pv:', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH')

In [10]:
assert similar is None
assert similar1 is None

### Combine spaces and other characters

In [11]:
print(forge.resolve('229_l6 it ctx', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH'))

{
    id: https://bbp.epfl.ch/ontologies/core/ttypes/229_L6_IT_CTX
    type: Class
    label: 229_L6 IT CTX
    subClassOf:
    [
        https://bbp.epfl.ch/ontologies/core/ttypes/L4_5_6_IT_Car3
        https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType
    ]
}


In [12]:
print(forge.resolve('264_L5/6 np CTX', scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH'))

{
    id: https://bbp.epfl.ch/ontologies/core/ttypes/264_L5_6_NP_CTX
    type: Class
    label: 264_L5/6 NP CTX
    subClassOf:
    [
        https://bbp.epfl.ch/ontologies/core/ttypes/L5_6_NP_CTX
        https://bbp.epfl.ch/ontologies/core/celltypes/NeuronTranscriptomicType
    ]
}


## Resolve from a specific ontology
This is particularly useful when two entities have a similar acronym like the example below, where there is a brain region called BAC, and a electrical cell type labelled bAC.

Without any specification, it will return the first result in the list of matches (no priority or ranking).

### List available ontologies to resolve from

In [13]:
list(forge.resolvers(output='dict')['ontology'].keys()) # terms is for resolving accross all ontologies

['terms', 'CellType', 'BrainRegion', 'Species']

In [14]:
from kgforge.core.commons.strategies import ResolvingStrategy

print(forge.resolve("bAC", scope="ontology", target="terms", 
                    strategy=ResolvingStrategy.EXACT_CASEINSENSITIVE_MATCH))

{
    id: http://uri.interlex.org/base/ilx_0738199
    type: Class
    label: bAC
    notation: bAC
    prefLabel: Burst accommodating electrical type
    subClassOf:
    [
        nsg:EType
        bmo:NeuronElectricalType
    ]
}


In [15]:
print(forge.resolve("bAC", scope="ontology", target="CellType", strategy="EXACT_CASEINSENSITIVE_MATCH"))

{
    id: http://uri.interlex.org/base/ilx_0738199
    type: Class
    label: bAC
    notation: bAC
    prefLabel: Burst accommodating electrical type
    subClassOf:
    [
        nsg:EType
        bmo:NeuronElectricalType
    ]
}


In [16]:
print(forge.resolve("bAC", scope="ontology", target="BrainRegion", strategy="EXACT_CASEINSENSITIVE_MATCH"))

{
    id: http://api.brain-map.org/api/v2/data/Structure/287
    type: Class
    label: Bed nucleus of the anterior commissure
    isDefinedBy: http://bbp.epfl.ch/neurosciencegraph/ontologies/core/brainregion
    notation: BAC
    prefLabel: Bed nucleus of the anterior commissure
    subClassOf: nsg:BrainRegion
}


In [17]:
print(forge.resolve("rat", scope="ontology", target="Species", strategy="EXACT_CASEINSENSITIVE_MATCH"))

{
    id: http://purl.obolibrary.org/obo/NCBITaxon_10114
    type: Class
    label: Rat
    subClassOf: obo:NCBITaxon_9989
}


### Ensuring one label

In [18]:
resource = forge.resolve("AOB_MC", scope="ontology", target="terms", strategy="EXACT_MATCH")
assert isinstance(resource.label, str)

In [19]:
excitatory = forge.resolve("exc", scope='ontology', strategy='EXACT_CASEINSENSITIVE_MATCH')
assert isinstance(excitatory.label, str)

In [20]:
print(excitatory)

{
    id: https://neuroshapes.org/ExcitatoryNeuron
    type: Class
    label: Excitatory Neuron
    notation: Exc
    prefLabel: Excitatory Neuron
    subClassOf: bmo:NeurotransmitterType
}
